In [115]:
import torch
from torch import nn
import pandas as pd
import torch.optim as optim
import torch.nn.functional as F
from torch.nn.utils.rnn import pad_sequence
import copy
from torch.utils.data import Dataset, DataLoader
import gc
import random



In [116]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# from numba import cuda
# device = cuda.get_current_device()
# device.reset()
print(device)

cuda


In [117]:
# train_csv = "/kaggle/input/aksh11/aksharantar_sampled/tel/tel_train.csv"
# test_csv = "/kaggle/input/aksh11/aksharantar_sampled/tel/tel_test.csv"
# val_csv = "/kaggle/input/aksh11/aksharantar_sampled/tel/tel_valid.csv"
train_csv = "/kaggle/input/aksh11/aksharantar_sampled/hin/hin_train.csv"
test_csv = "/kaggle/input/aksh11/aksharantar_sampled/hin/hin_test.csv"
val_csv = "/kaggle/input/aksh11/aksharantar_sampled/hin/hin_valid.csv"

In [118]:
train_df = pd.read_csv(train_csv, header=None)
train_source, train_target = train_df[0].to_numpy(), train_df[1].to_numpy();
test_df = pd.read_csv(test_csv, header=None)
val_df = pd.read_csv(val_csv, header=None)


In [119]:
start_token = "<"
end_token  = ">"
def get_chars(str, char_index_dict):
    chars_indexes = []
    for ch in str:
        chars_indexes.append(char_index_dict[ch])
    return torch.tensor(chars_indexes, device=device)

def preprocess_data(source_data, target_data):
    data = {
            "source_chars" : ["_"],
            "target_chars" : ["_"],
            "source_char_index" : {"_":0},
            "source_index_char" : {0:"_"},
            "target_char_index" :{"_":0},
            "target_index_char" : {0:"_"},
            "source_len":0,
            "target":0,
            "source_data":source_data,
            "target_data":target_data,
            "source_data_seq":[],
            "target_data_seq":[]
           }
    for i in range(len(source_data)):
        source_data[i] = start_token + source_data[i] + end_token
        target_data[i] = start_token + target_data[i] + end_token
        
        
        for c in source_data[i]:
            if(data["source_char_index"].get(c) == None):
                data["source_chars"].append(c)
                idx = len(data["source_chars"]) -1
                data["source_char_index"][c] = idx
                data["source_index_char"][idx] = c
        for c in target_data[i]:
            if(data["target_char_index"].get(c) == None):
                data["target_chars"].append(c)
                idx = len(data["target_chars"]) -1
                data["target_char_index"][c] = idx
                data["target_index_char"][idx] = c
        data["source_data_seq"].append(get_chars(source_data[i], data['source_char_index']))
        data["target_data_seq"].append(get_chars(target_data[i], data['target_char_index']))
        
        
    data['source_data_seq'] = pad_sequence(data['source_data_seq'], batch_first=True, padding_value=0)
    data['target_data_seq'] = pad_sequence(data['target_data_seq'], batch_first=True, padding_value=0)
    print(data['source_data_seq'].shape)
    print(data['source_data_seq'])
    print(data['target_data_seq'].shape)
    print(data['target_data_seq'])
#     data['source_data_seq'] = torch.tensor(data['source_data_seq'])
#     data['target_data_seq'] = torch.tensor(data['target_data_seq'])
#     print(data['source_data_seq'][0])
#     print(data['target_data_seq'][0])
#     print(data['source_data'][0])
#     print(data['target_data'][0])
      
    data["source_len"] = len(data["source_chars"])
    data["target_len"] = len(data["target_chars"])
    return data;d


data = preprocess_data(copy.copy(train_source), copy.copy(train_target[:]))
print(data["source_chars"])
print(data["target_chars"])
print(data["source_len"])
print(data["target_len"])

torch.Size([51200, 26])
tensor([[ 1,  2,  3,  ...,  0,  0,  0],
        [ 1,  9, 10,  ...,  0,  0,  0],
        [ 1, 14, 10,  ...,  0,  0,  0],
        ...,
        [ 1,  4,  2,  ...,  0,  0,  0],
        [ 1,  2, 21,  ...,  0,  0,  0],
        [ 1,  4, 11,  ...,  0,  0,  0]], device='cuda:0')
torch.Size([51200, 22])
tensor([[ 1,  2,  3,  ...,  0,  0,  0],
        [ 1, 10, 11,  ...,  0,  0,  0],
        [ 1, 15, 11,  ...,  0,  0,  0],
        ...,
        [ 1, 40,  3,  ...,  0,  0,  0],
        [ 1,  3, 30,  ...,  0,  0,  0],
        [ 1, 40, 17,  ...,  0,  0,  0]], device='cuda:0')
['_', '<', 's', 'h', 'a', 't', 'r', 'g', '>', 'b', 'i', 'n', 'd', 'y', 'k', 'o', 'p', 'v', 'e', 'c', 'm', 'u', 'w', 'l', 'j', 'x', 'f', 'z', 'q']
['_', '<', 'श', 'स', '्', 'त', 'र', 'ा', 'ग', '>', 'ब', 'ि', 'न', 'द', 'य', 'क', 'ण', 'ं', 'ज', 'ञ', 'ो', 'प', 'व', 'ी', 'ट', 'च', 'े', 'भ', 'म', 'ध', 'ु', 'घ', 'ड', '़', 'ह', 'ल', 'ै', 'इ', 'ॉ', 'ू', 'अ', 'ए', 'ौ', 'आ', 'ई', 'झ', 'ः', 'ख', 'ष', 'उ', 'थ', 'छ', 'ठ'

In [120]:
char_embd_dim=128
input_dim = data["source_len"]
output_dim = data["target_len"]
hidden_layer_neurons = 512
learning_rate  =0.0001
batch_size = 32
number_of_layers = 10
tf_ratio = 0.5

In [121]:
class Encoder(nn.Module):
    def __init__(self, ):
        super(Encoder, self).__init__()
        self.embedding = nn.Embedding(input_dim, char_embd_dim)
        self.rnn = nn.GRU(char_embd_dim, hidden_layer_neurons,num_layers=number_of_layers, batch_first=True)
        
    def forward(self, current_input, prev_state):
        embd_input = self.embedding(current_input)
        output, prev_state = self.rnn(embd_input, prev_state)
        return output, prev_state
    def getInitialState(self):
        return torch.zeros(number_of_layers,batch_size,hidden_layer_neurons, device=device)

    
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.embedding = nn.Embedding(output_dim, char_embd_dim)
        self.rnn = nn.GRU(char_embd_dim, hidden_layer_neurons,num_layers=number_of_layers, batch_first=True)
        self.fc = nn.Linear(hidden_layer_neurons, output_dim)
        self.softmax = nn.LogSoftmax(dim=2)

    def forward(self, current_input, prev_state):
        embd_input = self.embedding(current_input)
        curr_embd = F.relu(embd_input)
        output, prev_state = self.rnn(curr_embd, prev_state)
        output = self.softmax(self.fc(output))
        return output, prev_state

In [122]:
class MyDataset(Dataset):
    def __init__(self, data):
        self.source_data_seq = data[0]
        self.target_data_seq = data[1]
    
    def __len__(self):
        return len(self.source_data_seq)
    
    def __getitem__(self, idx):
        source_data = self.source_data_seq[idx]
        target_data = self.target_data_seq[idx]
        return source_data, target_data


In [123]:
h_params  ={
    "learning_rate":0.0001,
    "epochs":30
}

def make_strings(data, source, target, output):
    source_string = ""
    target_string = ""
    output_string = ""
#     print(output)
    for i in source:
#         print(i.item())
        source_string+=(data['source_index_char'][i.item()])
    for i in target:
        target_string+=(data['target_index_char'][i.item()])
    for i in output:
        output_string+=(data['target_index_char'][i.item()])
    return source_string, target_string, output_string
                        

def train(data, h_params):
    encoder = Encoder().to(device)
    decoder = Decoder().to(device)
    
    encoder_optimizer = optim.Adam(encoder.parameters(), lr=h_params["learning_rate"])
    decoder_optimizer = optim.Adam(decoder.parameters(), lr=h_params["learning_rate"])
    
    loss_fn = nn.NLLLoss()
    training_data = [data["source_data_seq"], data['target_data_seq']]
    my_dataset = MyDataset(training_data)
    data_loader = DataLoader(my_dataset, batch_size=batch_size, shuffle=True)
    
    for ep in range(h_params["epochs"]):
        for batch_num, (source_batch, target_batch) in enumerate(data_loader):
#             if(batch_num>0):
#                 break
            encoder_initial_state = encoder.getInitialState()
            encoder_output, encoder_current_state = encoder(source_batch,encoder_initial_state)
#             print(encoder_current_state)
            loss = 0
            correct = 0
            decoder_curr_state = encoder_current_state
            output_seq_len = target_batch.shape[1]
            decoder_actual_output = []
#             print(target_batch)
            
            use_teacher_forcing = True if random.random() < tf_ratio else False

            for i in range(output_seq_len):
#                 if(i>0):
#                     break
                if(i == 0):
                    dec_input_tensor = target_batch[:, i].view(batch_size,1)
#                     print(dec_input_tensor)
                curr_target_chars = target_batch[:, i]
                decoder_output, decoder_curr_state = decoder(dec_input_tensor,decoder_curr_state)
#                 print(decoder_output)
                topv, topi = decoder_output.topk(1)
#                 print("topv", topv)
                decoder_input_tensor = topi.squeeze().detach()
#                 print(curr_target_chars.shape, decoder_input_tensor.shape)
#                 print(curr_target_chars, decoder_input_tensor)
#                 print(target_batch.shape)
                decoder_actual_output.append(decoder_input_tensor)
                if(i<output_seq_len-1):
                    if use_teacher_forcing:
                        decoder_input_tensor = target_batch[:, i+1].view(batch_size, 1)
                    else:
                        decoder_input_tensor = decoder_input_tensor.view(batch_size, 1)

#                 print("dec", decoder_input_t)
    
                
                decoder_output = decoder_output[:, -1, :]
                
#                 print(decoder_output.shape)
            
                loss+=(loss_fn(decoder_output, curr_target_chars))
            
#             print(decoder_actual_output.shape)
#             print(decoder_actual_output)

            decoder_actual_output = torch.cat(decoder_actual_output,dim=0).view(output_seq_len, batch_size).transpose(0,1)
            
# #             print(decoder_actual_output.shape)
#             print(len(decoder_actual_output))
#             print(decoder_actual_output)
            if(batch_num == 0):
                    for j in range(20):
                        print(make_strings(data,source_batch[j],target_batch[j],decoder_actual_output[j]))
            
            correct = (decoder_actual_output == target_batch).all(dim=1).sum().item()
            if(batch_num%20 == 0):
                print("ep:", ep, " bt:", batch_num, " loss:", loss.item()/output_seq_len, " acc: ", correct/batch_size)
            encoder_optimizer.zero_grad()
            decoder_optimizer.zero_grad()
            loss.backward()
            encoder_optimizer.step()
            decoder_optimizer.step()
    encoder=None
    decoder=None
    gc.collect()
    torch.cuda.empty_cache()
        
train(data, h_params)

('<temptation>______________', '<टैम्पटेशन>___________', 'ौौऊनननननणणणणणणणणणणणणणण')
('<arjavsagar>______________', '<आर्जवसागर>___________', 'ौौऊनननननणणणणणणणणणणणणणण')
('<soochakankon>____________', '<सूचकांकों>___________', 'ौौऊनननननणणणणणणणणणणणणणण')
('<sublimated>______________', '<सब्लिमेटेड>__________', 'ौौऊनननननणणणणणणणणणणणणणण')
('<kalamassery>_____________', '<कलमस्सेरी>___________', 'ौौऊनननननणणणणणणणणणणणणणण')
('<sirnoo>__________________', '<सिरनू>_______________', 'ौरऊनननननणणणणणणणणणणणणणण')
('<vallis>__________________', '<वैलिस>_______________', 'ौरऊनननननणणणणणणणणणणणणणण')
('<banil>___________________', '<बनील>________________', 'ौरऊनननननणणणणणणणणणणणणणण')
('<kasbai>__________________', '<कस्बई>_______________', 'ौरऊनननननणणणणणणणणणणणणणण')
('<aapabako>________________', '<आपबको>_______________', 'ौौऊनननननणणणणणणणणणणणणणण')
('<nifty>___________________', '<निफ़्टी>_____________', 'ौरऊनननननणणणणणणणणणणणणणण')
('<jaslai>__________________', '<जसलाई>_______________', 'ौरऊनननननणणणणणणणणणणणणणण')
('<b